In [ ]:
import numpy as np
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


In [2]:
data = pd.read_csv('inputs.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [3]:
data = data[data.sentiment != "Neutral"]
print(data['text'][0:5])
data['text'] = data['text'].apply(lambda x: str(x).lower())
print("------------------")
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
print(data['text'][0:5])

# replace all special chars with null

print(data[ data['sentiment'] == 'pos'].size)
print(data[ data['sentiment'] == 'neg'].size)

i = 0
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
print(X)

0                                      not a good Idea
1                                             Horrible
2                                       LOVE LOVE LOVE
3    don't waste your money, pay more and buy one a...
4            leopard home button sticker for iphone 4s
Name: text, dtype: object
------------------
0                                      not a good idea
1                                             horrible
2                                       love love love
3    dont waste your money pay more and buy one at ...
4            leopard home button sticker for iphone 4s
Name: text, dtype: object
217326
26558
[[  0   0   0 ...,   7   8 332]
 [  0   0   0 ...,   0   0 246]
 [  0   0   0 ...,   9   9   9]
 ..., 
 [  0   0   0 ...,   0 261  32]
 [  0   0   0 ...,  19 161 752]
 [  0   0   0 ...,  87  37 732]]


In [4]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 30, 128)       256000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 196)           254800      embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 2)             394         lstm_1[0][0]                     
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
____________________________________________________________________________________________________
None


In [5]:
Y = pd.get_dummies(data['sentiment']).values # one hot
print(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

[[1 0]
 [1 0]
 [0 1]
 ..., 
 [0 1]
 [0 1]
 [0 1]]
(81701, 30) (81701, 2)
(40241, 30) (40241, 2)


In [6]:
batch_size = 32
model.fit(X_train, Y_train, nb_epoch = 1, batch_size=batch_size, verbose = 2)

Epoch 1/1
471s - loss: 0.1895 - acc: 0.9370


In [7]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]c
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.12
acc: 0.96


In [ ]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")